## Import packages to get your notebook to work!

In [2]:
# ---- import packages
import pandas as pd
import json
import os
import requests
import base64
import webbrowser

# ---- this will make the links that we print out clickable!
def make_clickable(val):
    return '<a href="{}">{}</a>'.format(val,val)

## Import your user researcher's Qualtrics IDs
[How to find the Qualtrics IDs on Qualtrics](https://www.qualtrics.com/support/integrations/api-integration/finding-qualtrics-ids/#LocatingQualtricsIDs)

In [4]:
# ---- import qualtrics ids
qualtrics_data_center = 'YOUR QUALTRICS DATA CENTER'
api_token = 'YOUR QUALTRICS API TOKEN'
survey_id = 'YOUR QUALTRICS SURVEY ID'

---
## Let's import the (made up) user data into our notebook

In [16]:
# ---- import user data
repo_path = os.path.dirname(os.path.abspath(''))
user_data = pd.read_excel(os.path.join(repo_path, 'data/user_data.xlsx'))

# ---- show top five lines of user data
user_data.head()

,email,market,device_type,monthly_active_user,account_type,sessions_in_app_30_days,searches_in_app_30_days
0,example_email0@outlook.com,Canada,android,1,paid,1,1
1,example_email1@icloud.com,US,android,1,free,17,0
2,example_email2@outlook.com,US,android,1,paid,4,4
3,example_email3@icloud.com,Canada,apple,1,paid,2,0
4,example_email4@gmail.com,US,android,0,paid,0,0


---
## Create the embedded data fields in the Qualtrics survey
To embed user data into a survey, you need to tell Qualtrics to look for it.  
Rather than [go to the website and type in the column headers by hand](https://www.qualtrics.com/support/survey-platform/survey-module/survey-flow/standard-elements/embedded-data/#CreatingAnEmbeddedDataElement), let's do it here.

In [10]:
# create embedded data fields in the Qualtrics survey (Qualtrics need to be expect that we are going to send user data)
# ---- reformat user data headers into embedded data fields json
user_data_headers = {'embeddedDataFields': [{'key': column_header} for column_header in user_data.columns]}

# ---- define url for post call
url = 'https://{}/API/v3/surveys/{}/embeddeddatafields'.format(qualtrics_data_center, survey_id)

# ---- post data to survey
requests.post(url, json = user_data_headers, headers={'x-api-token': api_token, 'content-type': 'application/json'})

<Response [200]>

- What does `<Response [200]>` mean? **Success!** Your columns headers were successfully turned into embedded data fields in your survey.
- I got `<Response [403]>`? **Permission Denied...** Your user researcher does not have permission to interact with the API. You will need to contact your IT administrator to get the setting on the license changed!
- I got `<Response [400]>`? **Attempt Failed...** Your user researcher has permission to interact with the API, but your request did not work. Every time I ran into this issue, it was because of a typo that meant I was not sending Qualtrics the data in the form that they expected.



## Let's go to the survey and make sure that it worked!

In [12]:
pd.DataFrame(
    ['https://{}/Q/EditSection/Blocks?ContextSurveyID={}'.format(qualtrics_data_center, survey_id)],
    columns = ['link to your survey'],
    index = ['click:'])\
.style.format(make_clickable)

,link to your survey
click:,https://YOUR QUALTRICS DATA CENTER/Q/EditSection/Blocks?ContextSurveyID=YOUR QUALTRICS SURVEY ID


---
## Create the unique survey links
When a person opens the survey using their unique link, their user data gets embedded.

In [17]:
# create unique survey links; when a user opens the survey link, it embeds their user data into the survey response
unique_links = []
for row in user_data.index:
    # ---- data needs to be base64 encoded
    encoded_user_data = base64.b64encode(bytes(user_data.loc[row].to_json(), 'utf-8')).decode('utf-8')
    # ---- encoded data gets appended to the end of the survey link using Qualtric's Q_EED function
    unique_links.append('https://{}/jfe/form/{}?Q_EED={}'.format(qualtrics_data_center, survey_id, encoded_user_data))
user_data['unique_survey_link'] = unique_links

# ---- export survey distribution list
email_distribution_list = user_data.set_index('email')[['unique_survey_link']]
email_distribution_list.to_excel(os.path.join(repo_path, 'data/survey_distribution_list.xlsx'))

# ---- show top 5 lines of email distribution list
email_distribution_list.head()

,unique_survey_link
email,
example_email0@outlook.com,https://YOUR QUALTRICS DATA CENTER/jfe/form/YO...
example_email1@icloud.com,https://YOUR QUALTRICS DATA CENTER/jfe/form/YO...
example_email2@outlook.com,https://YOUR QUALTRICS DATA CENTER/jfe/form/YO...
example_email3@icloud.com,https://YOUR QUALTRICS DATA CENTER/jfe/form/YO...
example_email4@gmail.com,https://YOUR QUALTRICS DATA CENTER/jfe/form/YO...


## Let's open the first link and make sure everything worked!

In [18]:
pd.DataFrame(
    ['{}'.format(email_distribution_list.unique_survey_link.values[0])],
    columns = ['link to your survey'],
    index = ['click:'])\
.style.format(make_clickable)

,link to your survey
click:,https://YOUR QUALTRICS DATA CENTER/jfe/form/YOUR QUALTRICS SURVEY ID?Q_EED=eyJlbWFpbCI6ImV4YW1wbGVfZW1haWwwQG91dGxvb2suY29tIiwibWFya2V0IjoiQ2FuYWRhIiwiZGV2aWNlX3R5cGUiOiJhbmRyb2lkIiwibW9udGhseV9hY3RpdmVfdXNlciI6MSwiYWNjb3VudF90eXBlIjoicGFpZCIsInNlc3Npb25zX2luX2FwcF8zMF9kYXlzIjoxLCJzZWFyY2hlc19pbl9hcHBfMzBfZGF5cyI6MX0=


---
---
---
## A brief warning -- the user data are not encrypted!
By placing user data in the link, anyone with knowledge of base64 encoding or the Qualtrics Q_EED function would be able to see their own user data.  
Below, I've decoded the first link:

In [10]:
base64.b64decode(email_distribution_list.unique_survey_link.values[0].split('Q_EED=')[1])

b'{"email":"example_email0@outlook.com","market":"Canada","device_type":"android","monthly_active_user":1,"account_type":"paid","sessions_in_app_30_days":1,"searches_in_app_30_days":1}'

If this violates your data handling policy, make sure that you encrypt the user data before you embed the column headers in the survey.  
It is important to do it before embedding the column headers in the survey because:
- If the column headers are unencrypted (email, market etc.)
- But our data are encrypted (01242, 98231 etc.)
- The the data keys will not match with the fields that Qualtrics expects and it will not work!